In [1]:
#General Functions

def approveChoice(InputQuestionString, AcceptableAnswerList):
    works = False
    while works == False:
        answer = input(InputQuestionString)
        if answer in AcceptableAnswerList:
            works = True
        else:
            print('Not a valid input')
            works = False
    return answer

def WLtoIndex(wl, w):
    details = min(enumerate(wl.iloc[:,0]), key=lambda x: abs(x[1]-w))
    return details[0]

def polyfit(SSorTR, df, wl, order, wlLow, wlHigh):
    k = WLtoIndex(wl, wlLow)
    j = WLtoIndex(wl, wlHigh)
    if SSorTR == 'SS':
        return ss_polyfit(df,wl,order,j,k)
    elif SSorTR == 'TR':
        return tr_polyfit(df,wl,order,j,k)
    else:
        return 'SSorTR'

def tr_polyfit(df, wl, order, j, k):
    wl_full = wl.values.T[0]
    h = 0
    l = df.shape[1]
    wl = wl_full[h:l]
    wvn1 = wl_full[h:j]
    wvn2 = wl_full[k:l]
    wvn = np.append(wvn1,wvn2)
    new_list = []
    for item in wl:
        new_list = np.append(new_list, str(item))
    
    new_df = pd.DataFrame()
    
    for m in np.arange(len(df.index)):
        data2 = df.iloc[m, h:l].to_numpy()
        dataA = df.iloc[m, h:j]
        dataB = df.iloc[m, k:l]
        data3 = np.append(dataA,dataB)

        coeff = np.polyfit(wvn, data3, order)
        correction = np.polyval(coeff, wl_full[h:l])
        data2c = data2 - correction
        new_df = new_df.append(pd.DataFrame(data2c).T)
    return new_df

def ss_polyfit(df, wl, order, j, k):
    h = 0
    l = len(df)
    wl = wl.iloc[h:l, 0]
    wvn1 = wl.iloc[h:j]
    wvn2 = wl.iloc[k:l]
    wvn = np.append(wvn1,wvn2)
    data2 = df.iloc[h:l]
    dataA = df.iloc[h:j]
    dataB = df.iloc[k:l]
    data3 = np.append(dataA,dataB)
    
    coeff = np.polyfit(wvn, data3, order)
    correction = np.polyval(coeff, wl[h:l])
    data2c = data2 - correction
    return data2c, correction

def fit_exponential_model(Peaks, time, with_plot, with_report, no_of_terms, **kwargs):
    mod = ExponentialModel()
    
    x = time
    y = Peaks

    def test_func(x, A1, A2, tau, **kwargs):
        if 'A3' in kwargs and 'tau2' in kwargs:
            return A2*np.exp(-x/tau) + kwargs.get('A2')*np.exp(-x/kwargs.get('tau2'))+A1
        else:
            return A2*np.exp(-x/tau) +A1

    if no_of_terms == 1: 
        mod = Model(test_func)
        out = mod.fit(y, x=x, A1=2221, tau=10*60, A2=7.5)
        params = [out.values['A1'], out.values['A2'], out.values['tau']]

    elif no_of_terms == 2: 
        print('terms = 2')
        mod = Model(test_func)
        out = mod.fit(y, x=x, A1=2221, tau=3*60, A2 = 1.5, A3=3.5, tau2=(21.06*60))
        params = [out.values['A1'], out.values['A2'], out.values['tau'], 
                  out.values['A3'], out.values['tau2']]

    if with_report == True:
        print(out.fit_report(min_correl=0.25))
        out.plot_residuals('k.')
        
    if with_plot == True:
        plt.figure()
        fig, ax = plt.subplots(figsize = (5,3))
        plt.plot(time/60, out.best_fit, 'r-', label='best fit')
        #plt.xlim(0, 90)

    return params

"""
Returns a dictionary of the amplitude, center, and sigma for the beginning and end of a specific identifier.
"""
def getUniversalVariables(identifier, SteadyStateSpectra):
    paramsDict = {}
    for key in SteadyStateSpectra[identifier].keys():
        spectrum = SteadyStateSpectra[identifier][key]
        y = np.zeros(len(spectrum))
        x = np.zeros(len(spectrum))
        for i in range(len(spectrum)):
            y[i] = spectrum.iloc[i,1]
            x[i] = spectrum.iloc[i,0]
        data = np.column_stack([x,y])
        dat = data
        x = dat[:, 0]
        y = dat[:, 1]

        mod = PseudoVoigtModel()
        pars = mod.guess(y, x=x)

        init = mod.eval(pars, x=x)
        out = mod.fit(y, pars, x=x)
        paramsDict[key] = out.best_values
    return paramsDict #returns a dictionary of amplitude, center, sigma


def GetStats(dictionary):
    List = []
    for key in dictionary:
        List.append(dictionary[key])
    mean = np.mean(List)
    std = np.std(List)
    return mean, std

In [2]:
#Functions for Collecting Data


"""
With user input, retrieves data to be used with Time Resolved functions.
"""
def getTimeResolvedRawData():
    works = False
    while works == False: 
        try:
            num = eval(input('How many conditions are you working with?: '))
            works = True
        except NameError:
            print("Must be an integer")
            works = False
            
   
    allRawSpectraDict = {}
    allRawData = {}

    for i in range(num):
        conditionSpectraDict = {}
        conditionDataDict = {}
        condition = input('What is the condition #' + str(i+1) + '?: ')
        works = False
        while works == False: 
            try:
                numSpectra = eval(input('How many time-resolved spectra are you entering for this condition?: '))
                works = True
            except NameError:
                print("Must be an integer")
                works = False
        for i in range(numSpectra):
            name = input('Enter name of replicate ' + str(i+1) + ': ')
            works = False
            while works == False:
                try:
                    loc = input('Enter location of replicate ' + str(i+1) + ': ')
                    spectrum, time, wl = ReadIn(loc)
                    data = {'spectrum':spectrum, 'time':time, 'wl':wl}
                    conditionSpectraDict[name] = spectrum
                    conditionDataDict[name] = data
                    works = True
                except FileNotFoundError:
                    print('Data not found. Try again.')
                    works = False
        allRawSpectraDict[condition] = conditionSpectraDict
        allRawData[condition] = conditionDataDict
    return allRawSpectraDict, allRawData


"""
With user input, retrieves spectra to be used with Steady State functions.
"""
def getAllSteadyStateDataForTR(dictionary):
    num = len(dictionary.keys())

    SteadyStateSpectra = {}
    for i in range(num):
        conditionSpectraDict = {}

        condition = list(dictionary.keys())[i]
        print('For the condition: '+str(condition))

        works = False
        while works == False: 
            try:
                begLoc = input('Enter location of beginning spectrum: ')
                begSpectrum = pd.read_table(begLoc, header=None)
                conditionSpectraDict['beg'] = begSpectrum
                works = True
            except FileNotFoundError:
                print('Beginning spectrum not found. Try again.')
                works = False
        works = False
        while works == False: 
            try:
                endLoc = input('Enter location of ending spectrum: ')
                endSpectrum = pd.read_table(endLoc, header=None)
                conditionSpectraDict['end'] = endSpectrum
                works = True
            except FileNotFoundError:
                print('Ending spectrum not found. Try again.')
                works = False
                
        SteadyStateSpectra[condition] = conditionSpectraDict
    return SteadyStateSpectra

"""
With user input, retrieves spectra to be used with Steady State functions.
"""
def getAllSteadyStateData():
    works = False
    while works == False: 
        try:
            num = eval(input('How many conditions are you working with?: '))
            works = True
        except NameError:
            print("Must be an integer")
            works = False
    SteadyStateSpectra = {}
    for i in range(num):
        conditionSpectraDict = {}
        condition = input('What is the condition #' + str(i+1) + '?: ')
        works = False
        while works == False: 
            try:
                begLoc = input('Enter location of beginning spectrum: ')
                begSpectrum = pd.read_table(begLoc, header=None)
                conditionSpectraDict['beg'] = begSpectrum
                works = True
            except FileNotFoundError:
                print('Beginning spectrum not found. Try again.')
                works = False
        works = False
        while works == False: 
            try:
                endLoc = input('Enter location of ending spectrum: ')
                endSpectrum = pd.read_table(endLoc, header=None)
                conditionSpectraDict['end'] = endSpectrum
                works = True
            except FileNotFoundError:
                print('Ending spectrum not found. Try again.')
                works = False
                
        SteadyStateSpectra[condition] = conditionSpectraDict
    return SteadyStateSpectra


"""
With user input, retrieves spectra to be used with Steady State functions.
"""
def getAllSteadyStateData_forSSonly():
    works = False
    while works == False: 
        try:
            num = eval(input('How many plots would you like to include? (up to 10): '))
            works = True
        except NameError:
            print("Must be an integer")
            works = False
    SteadyStateSpectra = {}
    for i in range(num):
        conditionSpectraDict = {}
        condition = input('What is the condition for plot #' + str(i+1) + '?: ')
        
        works = False
        while works == False: 
            try:
                numSpectra = eval(input('How many spectra would you like to overlay for plot #' + str(i+1) + '?: '))
                works = True
            except NameError:
                print("Must be an integer")
                works = False
        for j in range(numSpectra):
            replicate = input('Enter name of spectrum ' + str(j+1) + ': ')
            works = False
            while works == False: 
                try:
                    specLoc = input('Enter location of spectrum '+condition+' '+replicate+': ')
                    Spectrum = pd.read_table(specLoc, header=None)
                    conditionSpectraDict[replicate] = Spectrum
                    works = True
                except FileNotFoundError:
                    print('Spectrum '+condition+' '+replicate+'not found. Try again.')
                    works = False
                
        SteadyStateSpectra[condition] = conditionSpectraDict
    return SteadyStateSpectra

